# Bronze Layer – CRM CSV Ingestion

## Overview
This notebook ingests raw CSV files from a bronze volume into the bronze layer of the data lake.  
Each file is read from the volume, converted to delta format, and stored as a managed table in the `workspace.bronze` schema.

## Ingestion Configuration
The ingestion logic is driven by a configuration list (`files_properties`), where each entry defines:
- **source_system**: Origin system of the data (CRM or ERP)
- **table**: Target table name suffix
- **path**: File location in the bronze volume

This approach allows easy onboarding of new files without changing the ingestion code.


In [0]:
files_properties = [
    {
        "source_system": 'crm',
        "table": 'cust_info',
        "path": '/Volumes/workspace/bronze/source_systems/source_crm/cust_info.csv'
    },
    {
        "source_system": 'crm',
        "table": 'prd_info',
        "path": '/Volumes/workspace/bronze/source_systems/source_crm/prd_info.csv'
    },
    {
        "source_system":"crm",
        "table":"sales_details",
        "path":"/Volumes/workspace/bronze/source_systems/source_crm/sales_details.csv"
    },
    {
        "source_system": 'erp',
        "table": 'cust_az12',
        "path": '/Volumes/workspace/bronze/source_systems/source_erp/CUST_AZ12.csv'
    },
    {
        "source_system": 'erp',
        "table": 'loc_a101',
        "path": '/Volumes/workspace/bronze/source_systems/source_erp/LOC_A101.csv'
    },
    {
        "source_system":"erp",
        "table":"px_cat_g1v2",
        "path":"/Volumes/workspace/bronze/source_systems/source_erp/PX_CAT_G1V2.csv"
    }
]

In [0]:
for item in files_properties:
    print(f"Ingesting file '{item['source_system']}_{item['table']}' into table 'workspace.bronze.{item['source_system']}_{item['table']}'")
    
    df = spark.read \
        .format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(item['path'])

    df.write.mode("overwrite") \
        .format("delta") \
        .saveAsTable(f"workspace.bronze.{item['source_system']}_{item['table']}")